In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], "GPU")
import os
import shutil
from backend.database import Database
from backend.face_detector import FaceDetector
from backend.embedding_generator import EmbeddingGenerator
from PIL import Image
import numpy as np

2026-01-19 17:44:02.801465: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/danil/Documents/Project/MemeFaceMatcher/meme-comparator/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db = Database(path='../meme_storage/db.faiss')
fd = FaceDetector()
eg = EmbeddingGenerator()

No FAISS index was found. Initializing a new one!
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/danil/.insightface/models/buffalo_l/w600k_r50.onnx rec

INFO:root:Parsing model identifier. Schema: None, Identifier: ViT-B-32
INFO:root:Loaded built-in ViT-B-32 model config.
INFO:httpx:HTTP Request: HEAD https://huggingface.co/laion/CLIP-ViT-B-32-laion2B-s34B-b79K/resolve/main/open_clip_model.safetensors "HTTP/1.1 302 Found"
INFO:root:Instantiating model architecture: CLIP
INFO:root:Loading full pretrained weights from: /home/danil/.cache/huggingface/hub/models--laion--CLIP-ViT-B-32-laion2B-s34B-b79K/snapshots/1a25a446712ba5ee05982a381eed697ef9b435cf/open_clip_model.safetensors
INFO:root:Final image preprocessing configuration set: {'size': (224, 224), 'mode': 'RGB', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'interpolation': 'bicubic', 'resize_mode': 'shortest', 'fill_color': 0}
INFO:root:Model ViT-B-32 creation process complete.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
originals = sorted(os.listdir('../meme_storage'))
indexed = []
originals

['m1.jpeg',
 'm10.jpeg',
 'm11.jpeg',
 'm12.jpeg',
 'm13.jpeg',
 'm14.jpeg',
 'm2.jpeg',
 'm3.jpeg',
 'm4.jpeg',
 'm5.jpeg',
 'm6.jpeg',
 'm7.jpeg',
 'm8.jpeg',
 'm9.jpeg']

In [4]:
for file in originals:
    print(file)

    img = np.asarray(Image.open('../meme_storage/' + file))

    face_data = fd.detect(img)
    if face_data is None:
        indexed.append(None)
        continue
    else:
        cropped, emb = face_data

    emb, _ = eg.generate_embedding(cropped)

    indexed.append(db.add_vector(emb)[0])

m1.jpeg
m10.jpeg
m11.jpeg
m12.jpeg
m13.jpeg
m14.jpeg
m2.jpeg
No faces were found!
m3.jpeg
m4.jpeg
m5.jpeg
m6.jpeg
No faces were found!
m7.jpeg
m8.jpeg
m9.jpeg


In [5]:
indexed

[np.int64(0),
 np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 None,
 np.int64(6),
 np.int64(7),
 np.int64(8),
 None,
 np.int64(9),
 np.int64(10),
 np.int64(11)]

In [6]:
for original_path, id in zip(originals, indexed):
    if id is None:
        continue

    shutil.copy('../meme_storage/' + original_path, '../meme_storage/' + str(id) + '.jpeg')

In [7]:
del db # to save the index